In [1]:
pip install rembg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.2 MB/s eta 0:00:00


In [9]:
# 背景変更パート
from PIL import Image
from rembg import remove

img = Image.open("input.jpg") # 画像読み込み
bgless_img = remove(img) # 背景を削除

# このif文で透過部分を指定色(白)に置換
if bgless_img.mode in ('RGBA', 'LA'):
    background = Image.new(bgless_img.mode[:-1], bgless_img.size, "#FFFFFF")
    background.paste(bgless_img, bgless_img.split()[-1])
    bgless_img = background

bgless_img.save("output.png") # 背景を削除して白くしただけの画像を一旦保存

In [10]:
# 256階調から3階調にするパート
import numpy as np

img = np.array(Image.open('output.png').convert('L')) # 画像の読み込み

#量子化レベルを (1/q)　とする
q = 3 # 階調数

h, w = img.shape[:2] #　画像サイズを取得

#全画素を走査し,量子化レベルを変換
for x in range(h):
    for y in range(w):
        for l in range(q):
            t1 = l*((255)/q);
            t2 = (l+1)*((255)/q);

            # ポスタリゼーション処理
            if t1 <= img[x,y] and img[x,y] < t2:
                img[x,y] = (t1+t2)/2
                break

Image.fromarray(img).save('3kaicho.png') # 3階調にした画像を一旦保存

In [11]:
# 黒以外を黄色にするパート
import cv2

img = cv2.imread('3kaicho.png') # 画像の読み込み

# 変えたい色
target_color1 = (212, 212, 212)
target_color2 = (255, 255, 255) # 背景は白にしたので
target_color3 = (127, 127, 127)

# 変更後の色
change_color1 = (212, 212, 0)
change_color2 = (238, 238, 0) # カラーピッカーで背景色はこれにした
change_color3 = (238, 206, 0)

# 色の変更
# 画像サイズh，wの取得は前のパートでやっている
for i in range(h):
    for j in range(w):
        b, g, r = img[i, j]
        if (b, g, r) == target_color1:
            img[i, j] = change_color2
        elif (b, g, r) == target_color2:
            img[i, j] = change_color2
        elif (b, g, r) == target_color3:
            img[i, j] = change_color3
        else:
            img[i, j] = (0, 0, 0) # 髪とか目を想定

Image.fromarray(img).save('result.png') # 最終結果画像の保存


# colabでやっていて，いちいち保存した画像を確認するのが面倒な人へ
# from matplotlib import pylab as plt
# plt.imshow(img)
# plt.show()